In [ ]:
from summer2.functions import time as stf
import pandas as pd
import numpy as np

In [ ]:
pd.options.plotting.backend = "plotly"

In [ ]:
act_trial_screening_rate = {
        2014.0: 0.0,  # Value for 2014
        2014.1: 0.81,
        2015.0: 0.81,  # Value for 2015
        2015.1: 0.70,
        2016.0: 0.70,  # Value for 2016
        2016.1: 0.66,  # Value for 2017
        2017.0: 0.66,
        2017.1: 0.60,
        2018.0: 0.60,  # Value for 2018
        2018.1: 0.0,  # Value for 2019
}

In [ ]:
f_sig = stf.get_linear_interpolation_function(list(act_trial_screening_rate.keys()), list(act_trial_screening_rate.values()))
trial_year = np.linspace(2014,2020.5)
pd.Series(stf.get_time_callable(f_sig)(trial_year), index=trial_year).plot()

In [ ]:
act_control_screening_rate = {
        2017.0: 0.0,  # Value for 2017
        2017.1: 0.86,  # Value for 2017
        2018.0: 0.86,  # Value for 2018
        2018.1: 0.0,  # Value for 2019
    }

In [ ]:
f_sig = stf.get_sigmoidal_interpolation_function(list(act_control_screening_rate.keys()), list(act_control_screening_rate.values()), curvature=128)
control_year = np.linspace(2016,2020.5)
pd.Series(stf.get_time_callable(f_sig)(control_year), index=control_year).plot()